In [3]:
import tensorflow as tf

def make_const(shape, values, dtype=tf.float32, name=None):
    return tf.constant(values, shape=shape, dtype=dtype, name=name)

def make_int_const(shape, values, dtype=tf.int32, name=None):
    return tf.constant(values, shape=shape, dtype=dtype, name=name)

def squeeze_and_excite(input_tensor, channels, se_weights, name, cpu=False):
    with tf.name_scope(name):
        pooled = tf.reduce_mean(input_tensor, axis=[2, 3] if cpu else [1, 2], keepdims=True)
        w1 = make_const([channels, se_weights['se_channels']], se_weights['w1'])
        b1 = make_const([se_weights['se_channels']], se_weights['b1'])
        dense1 = tf.nn.relu(tf.nn.bias_add(tf.matmul(pooled, w1), b1))
        
        w2 = make_const([se_weights['se_channels'], 2 * channels], se_weights['w2'])
        b2 = make_const([2 * channels], se_weights['b2'])
        dense2 = tf.nn.bias_add(tf.matmul(dense1, w2), b2)
        scale = tf.sigmoid(dense2[:, :, :, :channels])
        excitation = scale * input_tensor
        
        return excitation

def make_conv_block(input_tensor, channels, input_channels, output_channels, weights, name, relu=True, seunit=None):
    with tf.name_scope(name):
        w_conv = make_const([channels, channels, input_channels, output_channels], weights['conv_weights'])
        conv2d = tf.nn.conv2d(input_tensor, w_conv, strides=[1, 1, 1, 1], padding='SAME')
        b_conv = make_const([output_channels], weights['conv_biases'])
        conv_with_bias = tf.nn.bias_add(conv2d, b_conv)
        
        if seunit:
            conv_with_bias = squeeze_and_excite(conv_with_bias, output_channels, seunit, name + "/se")
        
        if relu:
            return tf.nn.relu(conv_with_bias)
        else:
            return conv_with_bias

def make_residual_block(input_tensor, channels, weights, name, cpu=False):
    with tf.name_scope(name):
        conv1 = make_conv_block(input_tensor, 3, channels, channels, weights['conv1'], name + "/conv1", relu=True)
        conv2 = make_conv_block(conv1, 3, channels, channels, weights['conv2'], name + "/conv2", relu=False, seunit=weights.get('se', None))
        return tf.nn.relu(conv2 + input_tensor)


In [4]:
def test_residual_block():
    # Create a dummy input tensor with random data
    # Shape [batch_size, height, width, channels]
    input_tensor = tf.random.normal([1, 64, 64, 3])

    # Define dummy weights for the convolutional layers
    weights = {
        'conv1': {
            'conv_weights': tf.random.normal([3, 3, 3, 3]),
            'conv_biases': tf.random.normal([3]),
        },
        'conv2': {
            'conv_weights': tf.random.normal([3, 3, 3, 3]),
            'conv_biases': tf.random.normal([3]),
            'se': {
                'se_channels': 4,
                'w1': tf.random.normal([3, 4]),
                'b1': tf.random.normal([4]),
                'w2': tf.random.normal([4, 6]),
                'b2': tf.random.normal([6]),
            }
        }
    }

    # Run the residual block function
    output = make_residual_block(input_tensor, 3, weights, "test_block")

    # Print the output shape and some details to verify correctness
    print("Output shape:", output.shape)
    print("Output tensor:", output)

# Run the test function
if __name__ == "__main__":
    test_residual_block()

Output shape: (1, 64, 64, 3)
Output tensor: tf.Tensor(
[[[[ 0.          1.7313688   0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   ...
   [ 0.          0.          0.        ]
   [ 1.7498739   0.          0.        ]
   [ 0.          0.5159397   0.        ]]

  [[ 0.          7.5680194   0.        ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]
   ...
   [ 0.         11.978024   21.85036   ]
   [ 0.          0.          0.        ]
   [ 0.          0.          0.        ]]

  [[14.911353    7.839838    0.        ]
   [ 0.          0.          0.        ]
   [ 0.         23.082615    0.6489618 ]
   ...
   [ 0.          0.          0.        ]
   [ 3.7793534   0.          0.        ]
   [ 0.72525287  0.         16.881302  ]]

  ...

  [[10.655182    0.          0.        ]
   [ 0.         17.910152   26.609123  ]
   [ 0.         16.996769    0.        ]
   ...
   [ 0.          0.          0.        ]
  

In [3]:
%pwd

'c:\\Users\\rafal\\Python projects\\chess_xd\\chess\\lczero_training2\\lczero-training\\tf2'

In [5]:
!python net_to_model.py --cfg "configs/maia_config.yaml" "maia-1800.pb.gz"

dataset:
  input_test: PATH_TO_VAL_FILES
  input_train: PATH_TO_TRAINING_FILES
gpu: 0
model:
  filters: 64
  residual_blocks: 6
  se_ratio: 8
name: kb1-64x6
training:
  batch_size: 1024
  checkpoint_steps: 10000
  lr_boundaries:
  - 80000
  - 200000
  - 360000
  lr_values:
  - 0.1
  - 0.01
  - 0.001
  - 0.0001
  num_batch_splits: 1
  path: C:\Users\rafal\Python projects\chess_xd\chess\lczero_training2\lczero-training\networks
  policy_loss_weight: 1.0
  precision: half
  shuffle_size: 250000
  test_steps: 2000
  total_steps: 400000
  train_avg_report_steps: 50
  value_loss_weight: 1.0

Wrote model to C:\Users\rafal\Python projects\chess_xd\chess\lczero_training2\lczero-training\networks\kb1-64x6\kb1-64x6-0


2024-06-08 20:30:57.770283: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cudart64_110.dll
2024-06-08 20:30:59.849237: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library nvcuda.dll
2024-06-08 20:31:00.326777: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3050 Laptop GPU computeCapability: 8.6
coreClock: 1.5GHz coreCount: 16 deviceMemorySize: 4.00GiB deviceMemoryBandwidth: 178.84GiB/s
2024-06-08 20:31:00.327223: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cudart64_110.dll
2024-06-08 20:31:00.332602: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cublas64_11.dll
2024-06-08 20:31:00.332707: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library c

In [5]:
%echo %PATH%


c:\Users\rafal\Python projects\chess_xd\chess\.venv\Scripts;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.2\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.2\libnvvp;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.5\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.5\libnvvp;C:\Program Files\NVIDIA\CUDNN\v9.2\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.3\bin;C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.3\libnvvp;C:\Windows\system32;C:\Windows;C:\Windows\System32\Wbem;C:\Windows\System32\WindowsPowerShell\v1.0\;C:\Windows\System32\OpenSSH\;C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common;C:\Program Files\NVIDIA Corporation\NVIDIA NvDLISR;C:\Program Files\Git\cmd;C:\Users\rafal\AppData\Local\Microsoft\WindowsApps;C:\Users\rafal\AppData\Local\Programs\Microsoft VS Code\bin;C:\Users\rafal\AppData\Local\Programs\MiKTeX\miktex\bin\x64\;C:\Program Files\protoc\bin;C:\Program Files\NVIDIA Corporation\Nsight Compute 2020.3.0\;


In [6]:
!python net.py -i "maia-1800.pb.gz"

Found .pb.gz network
Blocks: 6
Filters: 64
pblczero.Net.magic: 448
pblczero.Net.license: 
pblczero.EngineVersion.major: 0
pblczero.EngineVersion.minor: 21
pblczero.EngineVersion.patch: 0
pblczero.Format.weights_encoding: LINEAR16
pblczero.NetworkFormat.input: INPUT_CLASSICAL_112_PLANE
pblczero.NetworkFormat.output: OUTPUT_WDL
pblczero.NetworkFormat.network: NETWORK_SE_WITH_HEADFORMAT
pblczero.NetworkFormat.policy: POLICY_CONVOLUTION
pblczero.NetworkFormat.value: VALUE_WDL
pblczero.NetworkFormat.moves_left: MOVES_LEFT_NONE
pblczero.TrainingParams.training_steps: 400000
pblczero.TrainingParams.learning_rate: 9.999999747378752e-05
pblczero.TrainingParams.mse_loss: 0.20176962018013
pblczero.TrainingParams.policy_loss: 1.4583964347839355
pblczero.TrainingParams.accuracy: 52.52495574951172
pblczero.TrainingParams.lc0_params: 

Writing output to: maia-1800.txt.gz
saved as 'maia-1800.txt.gz' 4.83M


In [7]:
import tensorflow as tf

# List available GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpus))
for gpu in gpus:
    print(gpu)


Num GPUs Available:  1
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [1]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [8]:

import tensorflow as tf
from tfprocess import TFProcess 
import yaml

# Create a new model instance (ensure it has the same architecture as the saved model)
config_path = "configs/maia_config.yaml"
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

model = TFProcess(config)

# Load the weights from the checkpoint
checkpoint_path = "maia-1800.pb.gz"
model.init_net_v2()

model.replace_weights_v2(checkpoint_path)

model.model.save('mymodel2.h5', save_format='h5')
# Now the model's weights have been restored and it is ready to be used.



Error setting GPU: Physical devices cannot be modified after being initialized, falling back to CPU.
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3050 Laptop GPU, compute capability 8.6


c:\Users\rafal\anaconda3\envs\chessenv\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [9]:
from tfprocess import ApplySqueezeExcitation, ApplyPolicyMap

# Define a dictionary mapping the name of the custom layers to the actual layers
custom_objects_dict = {
    'ApplySqueezeExcitation': ApplySqueezeExcitation,
    'ApplyPolicyMap': ApplyPolicyMap
}

# Use the custom_objects argument to load the model
loaded_model = tf.keras.models.load_model('mymodel2.h5', custom_objects=custom_objects_dict)



In [10]:
import tensorflow as tf
import lc0_az_policy_map

# Assuming ApplySqueezeExcitation is defined somewhere in your script
class ApplySqueezeExcitation(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(ApplySqueezeExcitation, self).__init__(**kwargs)
    
    def build(self, input_shape):
        self.reshape_size = input_shape[1][1]

    def call(self, inputs):
        x = inputs[0]
        excited = inputs[1]
        gammas, betas = tf.split(tf.reshape(excited, [-1, self.reshape_size, 1, 1]), 2, axis=1)
        return tf.nn.sigmoid(gammas) * x + betas

class ApplyPolicyMap(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(ApplyPolicyMap, self).__init__(**kwargs)
        self.fc1 = tf.constant(lc0_az_policy_map.make_map())  # Ensure lc0_az_policy_map.make_map() is defined or available

    def call(self, inputs):
        h_conv_pol_flat = tf.reshape(inputs, [-1, 80 * 8 * 8])
        return tf.matmul(h_conv_pol_flat, tf.cast(self.fc1, h_conv_pol_flat.dtype))

# Loading the model with both custom layers
loaded_model = tf.keras.models.load_model('mymodel2.h5', custom_objects={
    'ApplySqueezeExcitation': ApplySqueezeExcitation,
    'ApplyPolicyMap': ApplyPolicyMap
})

loaded_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 112, 64)]    0                                            
__________________________________________________________________________________________________
reshape (Reshape)               (None, 112, 8, 8)    0           input_1[0][0]                    
__________________________________________________________________________________________________
input/conv2d (Conv2D)           (None, 64, 8, 8)     64512       reshape[0][0]                    
__________________________________________________________________________________________________
input/bn (BatchNormalization)   (None, 64, 8, 8)     256         input/conv2d[0][0]               
______________________________________________________________________________________________

In [11]:
for layer in loaded_model.layers:
    if isinstance(layer, tf.keras.layers.InputLayer):
        print(f"Input layer name: {layer.name}")
        print(f"Input shape: {layer.input_shape}")
        print(f"Input dtype: {layer.dtype}")

# Alternatively, you can directly access the model's input details
print(f"Model input shape: {loaded_model.input_shape}")
print(f"Model input dtype: {loaded_model.inputs[0].dtype}")

Input layer name: input_1
Input shape: [(None, 112, 64)]
Input dtype: float32
Model input shape: (None, 112, 64)
Model input dtype: <dtype: 'float32'>


In [1]:
input = '''0000000000000000000000000001000000000000010000011000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000000000000000000000100000000000000000000 1
0000000000000000000000000000000000000000000000000000000000001000 1
0000000000000000000000000000000000000000000001000000000000000000 1
0000000000000000000000000000000000000000000000000000000001000000 1
0000000011100000000000000000000100000000000000000000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000000000000000000000000000100000000000000 1
0001000100000000000000000000000000000000000000000000000000000000 1
0000000000001000000000000000000000000000000000000000000000000000 1
0100000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000001000000000000010000011000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000000000000000000000100000100000000000000 1
0000000000000000000000000000000000000000000000000000000000001000 1
0000000000000000000000000000000000000000000001000000000000000000 1
0000000000000000000000000000000000000000000000000000000001000000 1
0000000011100000000000000000000100000000000000000000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000000100000000000000000000000000000000000 1
0001000100000000000000000000000000000000000000000000000000000000 1
0000000000001000000000000000000000000000000000000000000000000000 1
0100000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000001000000000000010000011000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000000000000000000000100000100000000000000 1
0000000000000000000000000000000000000000000000000000000000100000 1
0000000000000000000000000000000000000000000001000000000000000000 1
0000000000000000000000000000000000000000000000000000000001000000 1
0000000011100000000000000000000100000000000000000000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000000100000000000000000000000000000000000 1
0001000100000000000000000000000000000000000000000000000000000000 1
0000000000001000000000000000000000000000000000000000000000000000 1
0100000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000001100000000000010000011000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000000000000000000000100000100000000000000 1
0000000000000000000000000000000000000000000000000000000000100000 1
0000000000000000000000000000000000000000000001000000000000000000 1
0000000000000000000000000000000000000000000000000000000001000000 1
0000000011100000000000000000000100000000000000000000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000100000000000000000000000000000000000000000000 1
0001000100000000000000000000000000000000000000000000000000000000 1
0000000000001000000000000000000000000000000000000000000000000000 1
0100000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000001000000010000010000011000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000000000000000000000100000100000000000000 1
0000000000000000000000000000000000000000000000000000000000100000 1
0000000000000000000000000000000000000000000001000000000000000000 1
0000000000000000000000000000000000000000000000000000000001000000 1
0000000011100000000000000000100100000000000000000000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000100000000000000000000000000000000000000000000 1
0001000100000000000000000000000000000000000000000000000000000000 1
0000000000001000000000000000000000000000000000000000000000000000 1
0100000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000001100000010000010000011000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000000000000000000000100000100000000000000 1
0000000000000000000000000000000000000000000000000000000000100000 1
0000000000000000000000000000000000000000000001000000000000000000 1
0000000000000000000000000000000000000000000000000000000001000000 1
0000000011100000000001000000000100000000000000000000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000100000000000000000000000000000000000000000000 1
0001000100000000000000000000000000000000000000000000000000000000 1
0000000000001000000000000000000000000000000000000000000000000000 1
0100000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000001000000011000010000011000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000000000000000000000100000100000000000000 1
0000000000000000000000000000000000000000000000000000000000100000 1
0000000000000000000000000000000000000000000001000000000000000000 1
0000000000000000000000000000000000000000000000000000000001000000 1
0000000011100000000001000000000100000000000000000000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000100000000000000000000000000000000000000000000 1
0001000100000000000000000000000000000000000000000000000000000000 1
0000000000001000000000000000000000000000000000000000000000000000 1
0100000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000001000000011000010000011000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000000000000000000000100000100000000000000 1
0000000000000000000000000000000000000000000000000000000000100000 1
0000000000000000000000000000000000000000000001000000000000000000 1
0000000000000000000000000000000000000000000000000000000001000000 1
0000000011100000000001000000000100000000000000000000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000100000000000000000000000000000000000000000000 1
0001000100000000000000000000000000000000000000000000000000000000 1
0000100000000000000000000000000000000000000000000000000000000000 1
0100000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 1
0000000000000000000000000000000000000000000000000000000000000000 0
0000000000000000000000000000000000000000000000000000000000000000 1
1111111111111111111111111111111111111111111111111111111111111111 1'''.split('\n')

data = [[float(value) for value in element.split(" ")[0]] for element in input]


print(len(data))
print(len(data[0]))
print(data)

112
64
[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 

In [15]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

In [24]:
import numpy as np

model_data = np.array(data)
print(model_data.shape)
model_data = np.expand_dims(model_data, axis=0)
print(model_data.shape)


prediction = loaded_model.predict([model_data])

print(prediction)

(112, 64)
(1, 112, 64)
[array([[ -5.746  ,  -3.064  ,  -2.979  ,  -1.75   ,  -1.854  ,  -1.568  ,
         -0.952  ,  -2.885  ,  -0.918  ,  -0.3582 ,  -4.28   ,  -0.677  ,
         -3.498  ,  -3.434  ,  -2.697  ,  -2.73   ,  -2.29   ,  -2.617  ,
         -1.386  ,  -2.041  ,  -1.076  ,  -0.695  ,  -0.08655,  -4.633  ,
         -4.67   ,  -2.932  ,  -1.732  ,  -1.308  ,  -1.691  ,  -1.365  ,
         -1.373  ,  -3.82   ,  -3.428  ,  -1.825  ,  -0.3796 ,  -5.16   ,
         -0.5215 ,  -4.38   ,  -3.92   ,  -3.014  ,  -3.223  ,  -2.947  ,
         -2.58   ,  -1.192  ,  -1.98   ,  -1.207  ,  -0.4258 ,  -3.547  ,
         -5.656  ,  -3.715  ,  -2.822  ,  -1.18   ,  -1.177  ,  -1.445  ,
         -1.1045 ,  -1.482  ,  -3.16   ,  -6.965  ,   1.612  ,  -4.133  ,
         -2.2    ,  -3.756  ,  -2.918  ,  -2.293  ,  -3.63   ,  -1.317  ,
         -3.037  ,  -0.884  ,  -2.72   ,  -0.27   ,  -2.287  ,  -1.898  ,
         -2.047  ,  -3.133  ,  -4.54   ,  -3.398  ,  -2.385  ,  -0.9375 ,
         -1.03